In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = 'Data'

In [ ]:
Image = []
for filename in os.listdir(path):
    # Check if the file is an image
    if filename.lower().endswith('.jpg'):
        # Construct full file path
        file_path = os.path.join(path, filename)
        # Read the image using OpenCV
        image = cv2.imread(file_path)
        # Check if the image was read correctly
        if image is not None:
            Image.append(image)
        else:
            print(f"Warning: Unable to read {file_path}")

In [ ]:
print(f"Images Loaded: {len(Image)}")

In [ ]:
for i, image in enumerate(Image):
    # Convert the image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding or edge detection (e.g., Canny edge detection)
    # Option 1: Using binary thresholding
    ret, thresh = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    img_contour = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 3)
   
    # Visualize the contoured image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Contoured Image {i+1}")
    plt.imshow(cv2.cvtColor(img_contour, cv2.COLOR_BGR2RGB))

# Show the plots
plt.show()


In [ ]:
for i, image in enumerate(Image):
    # Convert the image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (e.g., binary thresholding)
    ret, thresh = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours and hierarchy
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw contours up to level 2
    img_contour = image.copy()
    for j, contour in enumerate(contours):
        # Check if the contour is within level 2
        if hierarchy[0][j][3] == -1 or hierarchy[0][j][3] == 0:  
            cv2.drawContours(img_contour, [contour], -1, (0, 0, 255), 3)
        elif hierarchy[0][j][3] == 1:  # Child contour (level 1)
            cv2.drawContours(img_contour, [contour], -1, (0, 0, 255), 3)
        elif hierarchy[0][j][3] == 2:  # Grandchild contour (level 2)
            cv2.drawContours(img_contour, [contour], -1, (0, 0, 255), 3)

    # Visualize the contoured image with hierarchy
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Contoured Image {i+1} (Up to Level 2)")
    plt.imshow(cv2.cvtColor(img_contour, cv2.COLOR_BGR2RGB))

plt.show()

In [ ]:
for i, image in enumerate(Image):
    # Convert the image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (e.g., binary thresholding)
    ret, thresh = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours and hierarchy
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize a variable to track the maximum level of hierarchy
    max_level = 0
    
    # Iterate over the hierarchy to determine the maximum level
    for j in range(len(hierarchy[0])):
        level = 0
        parent = hierarchy[0][j][3]
        
        # Traverse up the hierarchy to find how deep this contour is
        while parent != -1:
            level += 1
            parent = hierarchy[0][parent][3]
        
        # Track the maximum level encountered
        max_level = max(max_level, level)
    
    # Print the maximum level of the hierarchy for the current image
    print(f"Image {i+1} - Maximum Contour Hierarchy Level: {max_level}")

    # Visualize the contoured image (optional)
    img_contour = image.copy()
    for j, contour in enumerate(contours):
        cv2.drawContours(img_contour, [contour], -1, (0, 255, 0), 3)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Contoured Image {i+1}")
    plt.imshow(cv2.cvtColor(img_contour, cv2.COLOR_BGR2RGB))

# Show the plots
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

for image in Image:
    # Load the image
    # image = Image[0]  # Uncomment if you're loading an image

    # Define the lower and upper bounds for black color
    # Black is typically considered as very low values in all B, G, R channels
    lower_black = np.array([0, 0, 0], dtype="uint8")
    upper_black = np.array([50, 50, 50], dtype="uint8")  # You can adjust these values

    # Create a mask for black color
    black_mask = cv2.inRange(image, lower_black, upper_black)

    # Highlight the black areas on the original image
    highlighted_black = image.copy()
    highlighted_black[black_mask != 0] = [0, 0, 255]  # Highlight black regions in red (BGR: 0, 0, 255)

    # Display the original and the highlighted images
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.subplot(1, 2, 2)
    plt.title("Black Color Detection (Highlighted in Red)")
    plt.imshow(cv2.cvtColor(highlighted_black, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.show()


In [ ]:
# Heighlightening only the borders of the table in the image
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = Image[1]  # Uncomment if you're loading an image

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image
contour_image = image.copy()

# Iterate over all contours
for contour in contours:
    # Approximate contour and calculate the absolute difference between contour length and approximate contour length
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # If the number of vertices in the approximated contour is 4, then we assume the contour is of a straight-lined shape
    if len(approx) == 4:
        cv2.drawContours(contour_image, [approx], -1, (0, 255, 0), 8)

# Display the original and the contour images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

plt.subplot(1, 2, 2)
plt.title("Straight Line Contour Detection")
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

plt.show()

In [ ]:
# Heighlightening only the borders of the table in the image
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
image = Image[1]  # Uncomment if you're loading an image

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image
contour_image = image.copy()

# Create an empty list to store the areas of the contours
areas = []

# Iterate over all contours
for contour in contours:
    # Calculate the area of the contour and add it to the list
    areas.append(cv2.contourArea(contour))

# Calculate the average area of the contours
average_area = sum(areas) / len(areas)

# Iterate over all contours again
for contour in contours:
    # If the area of the contour is greater than the average area, it is likely an overlapping contour
    if cv2.contourArea(contour) > average_area:
        
        # Approximate contour and calculate the absolute difference between contour length and approximate contour length
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # If the number of vertices in the approximated contour is 4, then we assume the contour is of a straight-lined shape
        if len(approx) == 4:
            cv2.drawContours(contour_image, [approx], -1, (0, 255, 0), 8)

# Display the original and the contour images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

plt.subplot(1, 2, 2)
plt.title("Overlapping Straight Line Contour Detection")
plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

plt.show()

In [ ]:
# Heighlightening only the borders of the table in the image
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images
images = Image  # Uncomment if you're loading a list of images

Countored_Images = []

os.makedirs("Contoured", exist_ok=True)

# Iterate over all images
for i, image in enumerate(images):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original image
    contour_image = image.copy()

    # Create an empty list to store the areas of the contours
    areas = []

    # Iterate over all contours
    for contour in contours:
        # Calculate the area of the contour and add it to the list
        areas.append(cv2.contourArea(contour))

    # Calculate the average area of the contours
    average_area = sum(areas) / len(areas)

    # Iterate over all contours again
    for contour in contours:
        # If the area of the contour is greater than the average area, it is likely an overlapping contour
        if cv2.contourArea(contour) > average_area:
            # Approximate contour and calculate the absolute difference between contour length and approximate contour length
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # If the number of vertices in the approximated contour is 4, then we assume the contour is of a straight-lined shape
            if len(approx) == 4:
                cv2.drawContours(contour_image, [approx], -1, (0, 255, 0), 8)

    # Display the original and the contour images
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Original Image {i+1}")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.subplot(1, 2, 2)
    plt.title(f"Overlapping Straight Line Contour Detection {i+1}")
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.show()

    Countored_Images.append(contour_image)
    cv2.imwrite(f"Contoured/Contoured_{i+1}.jpg", contour_image)

In [ ]:
# Heighlightening only the borders of the table in the image
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the images
images = Image  # Uncomment if you're loading a list of images

Countored_Images = []

os.makedirs("Contoured", exist_ok=True)

# Iterate over all images
for i, image in enumerate(images):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original image
    contour_image = image.copy()

    # Create an empty list to store the areas of the contours
    areas = []

    # Iterate over all contours
    for contour in contours:
        # Calculate the area of the contour and add it to the list
        areas.append(cv2.contourArea(contour))

    # Calculate the average area of the contours
    average_area = sum(areas) / len(areas)

    # Create a list to store the approximated contours
    approx_contours = []

    # Iterate over all contours again
    for contour in contours:
        # If the area of the contour is greater than the average area, it is likely an overlapping contour
        if cv2.contourArea(contour) > average_area:
            # Approximate contour and calculate the absolute difference between contour length and approximate contour length
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # If the number of vertices in the approximated contour is 4, then we assume the contour is of a straight-lined shape
            if len(approx) == 4:
                approx_contours.append(approx)

    # Join approximated contours that are on the same line within a tolerance of 10 pixels
    joined_contours = []
    for approx in approx_contours:
        for joined in joined_contours:
            if np.allclose(approx, joined, atol=100):
                joined = np.concatenate((joined, approx), axis=0)
                break
        else:
            joined_contours.append(approx)

    # Draw the joined contours on the image
    for contour in joined_contours:
        cv2.drawContours(contour_image, [contour], -1, (0, 0,255), 8)

    # Display the original and the contour images
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title(f"Original Image {i+1}")
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.subplot(1, 2, 2)
    plt.title(f"Overlapping Straight Line Contour Detection {i+1}")
    plt.imshow(cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for display

    plt.show()

    # Countored_Images.append(contour_image)
    # cv2.imwrite(f"Contoured/Contoured_{i+1}.jpg", contour_image)